# 所学技术应用

## langchain每个api的实践和体验

tools
1. 过滤课程tool：过滤选修，必修
2. 排序课程tool：输入用户的喜好，排序用户喜爱的课程
3. 插入用户名称和对应课程tool
4. 删除用户名称和对应课程tool
5. 用户选课后名称检查，模糊搜索和校正tool

agent行为
1. agent 根据需要选择课程，且将和用户描述比较相似的排在前面
2. agent 做好选课和删除课程名称匹配，用户不必说出课程全名或者名称可以是错的，agent会调用chain自动校正
3. agent 在检查选课和退选课名称后执行选课或退选操作

## 数据存储

存储课程

存储用户对应选择的课程

In [137]:
course_dict = {
    "required":[
        "software engineering",
        "machine learning",
        "math",
        "physical",
        "badminton"
    ],
    "elective":[
        "tennis",
        "table tennis",
        "football",
        "deep learning",
        "NLP"
    ]
}

user_number = '139'
    
# 格式："name":[]
user_select_course = {
    "139":["tennis","football"]
}

## 初始化llm

In [33]:
# 初始化llm
from langchain.chat_models import ChatOpenAI
import os
llm_model = "Qwen2.5-14B"
os.environ["OPENAI_API_KEY"] = "None"
os.environ["OPENAI_API_BASE"] = "http://10.58.0.2:8000/v1"
# llm_completion = OpenAI(model_name="Qwen2.5-14B") 
# llm_chat = OpenAIChat(model_name="Qwen2.5-14B")
llm = ChatOpenAI(temperature=0, model=llm_model)


In [174]:
from langchain.agents import tool
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


# 过滤课程tool，llm输入课程类型，过滤对应类型结果
@tool
def filter_course(course_type:str)->str:
    """
        return course list, use this for filtering courses,
        there two kinds of courses: required , elective
        the input is string type and the content should be required or elective or all
        if user have not asked which kinds of course the input should be all
        the function will return the course list
    """
    if course_type=='all':
        all  = [course_dict["required"],course_dict["elective"]]
        return all
    return course_dict[course_type]

# 排序用户喜爱的课程
@tool
def sort_course(user_input:str)->str:
    '''
        do not use this function if user didn't mention his preferences for courses.

        the input is string type and the content should only be the original input
        return sorted course list,if user mentioned the course user liked
        the function will return sorted course list
    '''
    prompt = ChatPromptTemplate.from_template("""
    extract all the course , and sort these course based on the user input
    the courses user liked should be sorted in the front

    the output should only contains the sorted courses with sequence number
                                            
    <<< USER INPUT >>>

    {input}

        <<< COURSE DICT >>>

    """+str(course_dict["required"]+course_dict["elective"]))
    llmChain = LLMChain(llm=llm,prompt=prompt)
    return llmChain.run(user_input)

# 检查用户提及的课程名称，修正课程
@tool
def check_course(user_input:str)->str:
    '''
        use this function before user want to quit some courses!
        do not use this function if user didn't decide to select courses or quit/delete courses.
        use this function when user decide to select some courses or quit or delete some courses and you need to check whether the course name user mentioned are correct.
        the input is string type and the content should only be the original input
        return corrected course list
        the function will return corrected course list
    '''
    prompt = ChatPromptTemplate.from_template("""
        extract the course name which user selected from the user input text ,and the course name may not correct
        output the correct course name based on the course list

        the output should only contains the corrected courses name                                             
        <<< USER INPUT >>>
        {input}

        <<< COURSE DICT >>>

    """+str(course_dict["required"]+course_dict["elective"]))
    llmChain = LLMChain(llm=llm,prompt=prompt)
    return llmChain.run(user_input)

@tool
def select_course(course:str)->str:
    '''
        use this function to record user's selection after checking the course name that user selected.
        the input is string type and the content is the courses that user want to select,
        and the  format should like this: course name 1, course name 2, course name 3 , ...
        the function will return operation info
        the function will return success or the course name that failed to select
    '''
    print("input-on-select:"+course)
    is_succeed = "success"
    all_course =  course_dict['required']+course_dict['elective']
    courses_input_lists = course.split(',')
    legal_selected_courses =[c.strip() for c in courses_input_lists if c.strip() in all_course]
    cc = [c.strip() for c in courses_input_lists if c.strip() not in legal_selected_courses]
    if len(cc)>=1:
        print("***:选择时，存在一门课名称非法:"+str(cc))
        is_succeed='failed'
    if user_number not in user_select_course:
        user_select_course[user_number]=[]
    user_select_course[user_number] += [c for c in legal_selected_courses if c not in user_select_course[user_number]]
    print(is_succeed)
    return is_succeed
     
@tool
def quit_course(course:str)->str:
    '''
        before use  this function , you must check where the course name is correct!
        use this function to record user's operation of quiting some courses after checking the course name that user mentioned.
        the input is string type and the content is the courses that user want to quit,
        and the format should like this: course name 1, course name 2, course name 3 , ...
        the function will return operation info
        the function will return success or the course name that failed to quit
    '''
    print("input-on-quit:"+course)
    is_succeed = "success"
    all_course =  course_dict['required']+course_dict['elective']
    courses_input_lists = course.split(',')
    legal_selected_courses =[c.strip() for c in courses_input_lists if c.strip() in all_course]
    cc = [c.strip() for c in courses_input_lists if c.strip() not in legal_selected_courses]
    if len(cc)>=1:
        print("***:退选时，存在一门课名称非法:"+str(cc))
        is_succeed='failed'
    if user_number not in user_select_course:
        user_select_course[user_number]=[]
    user_select_course[user_number] = [c for c in user_select_course[user_number] if c not in legal_selected_courses]
    print(is_succeed)
    return is_succeed
     
 
@tool
def view_selected_course(course:str)->str:
    '''
        use this function to show user selected courses.
        the input is string type and the content is always empty,
        the function will return a list of course name that user selected
    '''
    return user_select_course[user_number]
     
    

In [175]:
import langchain
langchain.debug = True
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

# tools = load_tools(['llm-math'],llm=llm)
tools = load_tools(["llm-math"], llm=llm) 

agent= initialize_agent(
    tools+[filter_course,sort_course,check_course,select_course,quit_course,view_selected_course], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

# 使用agent

In [133]:
agent.run("有哪些课程")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "有哪些课程"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "有哪些课程",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for courses.\n\nthe input is string type and the content should onl

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] [10.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我已经获得了课程列表，现在可以回答用户的问题。\nFinal Answer: 课程列表如下：\n\n必修课：软件工程、机器学习、数学、物理、羽毛球\n\n选修课：网球、乒乓球、足球、深度学习、自然语言处理",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "我已经获得了课程列表，现在可以回答用户的问题。\nFinal Answer: 课程列表如下：\n\n必修课：软件工程、机器学习、数学、物理、羽毛球\n\n选修课：网球、乒乓球、足球、深度学习、自然语言处理",
            "response_metadata": {
              "token_usage": {
                "prompt_tokens": 829,
                "total_tokens": 881,
                "completion_tokens": 52
              },
              "model_name": "Qwen2.5-14B",
            

'课程列表如下：\n\n必修课：软件工程、机器学习、数学、物理、羽毛球\n\n选修课：网球、乒乓球、足球、深度学习、自然语言处理'

In [134]:
agent.run("有哪些选修课程")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "有哪些选修课程"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "有哪些选修课程",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for courses.\n\nthe input is string type and the content should

'选修课程有：网球、乒乓球、足球、深度学习、自然语言处理。'

In [135]:
agent.run("我喜欢体育,有体育相关的课么")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "我喜欢体育,有体育相关的课么"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "我喜欢体育,有体育相关的课么",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for courses.\n\nthe input is string type and the 

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] [27.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Thought: 用户想了解是否有与体育相关的课程，我需要先查看所有课程，然后筛选出与体育相关的课程。\nAction:\n```\n{\n  \"action\": \"filter_course\",\n  \"action_input\": \"all\"\n}\n```\n",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Thought: 用户想了解是否有与体育相关的课程，我需要先查看所有课程，然后筛选出与体育相关的课程。\nAction:\n```\n{\n  \"action\": \"filter_course\",\n  \"action_input\": \"all\"\n}\n```\n",
            "response_metadata": {
              "token_usage": {
                "prompt_tokens": 640,
                "total_tokens": 693,
                "completion_tokens

'有的，我们有羽毛球、网球、乒乓球和足球等体育相关的课程。'

In [163]:
agent.run("我想选择machine lesing 和 badminto这两门课程")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "我想选择machine lesing 和 badminto这两门课程"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "我想选择machine lesing 和 badminto这两门课程",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for cours

'您已成功选择了machine learning和badminton这两门课程。'

In [173]:
agent.run("我想退选 badmin了")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "我想退选 badmin了"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "我想退选 badmin了",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for courses.\n\nthe input is string type and the cont

'用户已成功退选"badminton"课程。'

In [176]:
agent.run("目前为止我选了哪些课了？")

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "目前为止我选了哪些课了？"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "目前为止我选了哪些课了？",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nfilter_course: return course list, use this for filtering courses,\nthere two kinds of courses: required , elective\nthe input is string type and the content should be required or elective or all\nif user have not asked which kinds of course the input should be all\nthe function will return the course list\nsort_course: do not use this function if user didn't mention his preferences for courses.\n\nthe input is string type and the cont

'您目前选择了以下课程：tennis, machine learning。'

In [169]:
user_select_course['139']

['tennis', 'machine learning', 'badminton']